In [1]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [2]:
mlflow.set_experiment('Bow Vs TfIdf')

2025/06/27 16:47:57 INFO mlflow.tracking.fluent: Experiment with name 'Bow Vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://ms-mlflow-bucket/812426101598992135', creation_time=1751023078091, experiment_id='812426101598992135', last_update_time=1751023078091, lifecycle_stage='active', name='Bow Vs TfIdf', tags={}>

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [4]:
df = pd.read_csv('data_preprocessed.csv')

In [5]:
df.shape

(36607, 2)

In [7]:
df.isnull().sum()

clean_comment    123
category           0
dtype: int64

In [8]:
df.dropna(subset =['clean_comment'], inplace = True)

In [9]:
df.isnull().sum()

clean_comment    0
category         0
dtype: int64

In [10]:
df.duplicated().sum()

np.int64(241)

In [12]:
df.drop_duplicates(inplace = True)

In [13]:
df.shape

(36243, 2)

In [14]:
df.duplicated().sum()

np.int64(0)

In [15]:

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


2025/06/27 16:51:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 16:52:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135/runs/f0b7446f862d4ffe94f0e4c32894419d
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135


2025/06/27 16:52:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 16:53:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135/runs/d7bf68656b2f454691901fe1c0f43d50
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135


2025/06/27 16:54:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 16:54:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135/runs/54cab8ca671c433e8064f39facafc9a7
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135


2025/06/27 16:56:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 16:56:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135/runs/9e469c27a920451fbbddc0754e6f243e
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135


2025/06/27 16:57:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/27 16:57:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135/runs/b30d908446674f97a30a1441f874ad5b
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/812426101598992135
